<a href="https://colab.research.google.com/github/LujainAK/GP/blob/main/DataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Data Preprocessing Code**

###Import Libraries

In [60]:
from google.colab import drive
import pandas as pd
import os
import shutil
import json
import numpy as np
import random
import cv2
import csv
from keras.preprocessing.image import ImageDataGenerator

###Read the csv file and specify paths

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
csv_file = '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/full_df.csv'
image_directory = '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/preprocessed_images'
df = pd.read_csv(csv_file)

###Data Exploring

In [4]:
# Show the first 5 columns of the csv file
df.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [5]:
raw_data = df.drop(columns=['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O', 'labels'])

raw_data.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,filepath,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,../input/ocular-disease-recognition-odir5k/ODI...,"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,../input/ocular-disease-recognition-odir5k/ODI...,"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,../input/ocular-disease-recognition-odir5k/ODI...,"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,../input/ocular-disease-recognition-odir5k/ODI...,"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,../input/ocular-disease-recognition-odir5k/ODI...,"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [6]:
# Number of classes (diseases) in the dataset
raw_data["target"].unique().size

8

In [7]:
raw_data.shape

(6392, 10)

In [8]:
targets = np.array(raw_data["target"].apply(lambda x: json.loads(x)).tolist())


targets.shape

(6392, 8)

In [9]:
classes = { 0: "Normal",
            1: "Diabetes",
            2: "Glaucoma",
            3: "Cataract",
            4: "Age related Macular Degeneration",
            5: "Hypertension",
            6: "Pathological Myopia",
            7: "Other diseases/abnormalities"
          }

In [10]:
# Creating the dataset
data = np.sum(targets, axis=0)

classes_names = list(classes.values())
values = list(data)

In [11]:
raw_data["class_name"] = np.argmax(targets, axis=1).tolist()
raw_data["class_name"] = raw_data["class_name"].replace(classes)

raw_data.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,filepath,target,filename,class_name
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,../input/ocular-disease-recognition-odir5k/ODI...,"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg,Normal
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,../input/ocular-disease-recognition-odir5k/ODI...,"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg,Normal
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,../input/ocular-disease-recognition-odir5k/ODI...,"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg,Diabetes
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,../input/ocular-disease-recognition-odir5k/ODI...,"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg,Diabetes
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,../input/ocular-disease-recognition-odir5k/ODI...,"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg,Diabetes


In [12]:

processed_labels = raw_data[["ID", "filename", "class_name", "target"]]
processed_labels.to_csv('/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/labels_clean.csv', index=False)

processed_labels.head()

,ID,filename,class_name,target
0,0,0_right.jpg,Normal,"[1, 0, 0, 0, 0, 0, 0, 0]"
1,1,1_right.jpg,Normal,"[1, 0, 0, 0, 0, 0, 0, 0]"
2,2,2_right.jpg,Diabetes,"[0, 1, 0, 0, 0, 0, 0, 0]"
3,4,4_right.jpg,Diabetes,"[0, 1, 0, 0, 0, 0, 0, 0]"
4,5,5_right.jpg,Diabetes,"[0, 1, 0, 0, 0, 0, 0, 0]"


In [13]:

processed_labels.shape

(6392, 4)

###Split data into disease folders

In [14]:
# Iterate over each row in the DataFrame
for index, row in processed_labels.iterrows():
    image_name = row['filename']
    disease = row['class_name']

    # Create a folder for the disease if it doesn't exist
    disease_folder = os.path.join(image_directory, disease)
    if not os.path.exists(disease_folder):
        os.makedirs(disease_folder)

    # Move the image to the disease folder
    image_path = os.path.join(image_directory, image_name)
    new_image_path = os.path.join(disease_folder, image_name)
    train_image_path = os.path.join(image_directory, disease_folder, "training", image_name)
    test_image_path = os.path.join(image_directory, disease_folder, "testing", image_name)
    if(os.path.exists(image_path)):
      if not (os.path.exists(new_image_path) or os.path.exists(train_image_path) or os.path.exists(test_image_path)):
        shutil.move(image_path, new_image_path)

print("Images have been organized into disease folders in Google Drive.")

Images have been organized into disease folders in Google Drive.


In [15]:
def return_to_normal():
  normal_path = '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/preprocessed_images/Normal'
  for folder in ["training", "testing"]:
    tmppath = os.path.join(normal_path, folder)
    tmparr = os.listdir(tmppath)
    for img in tmparr:
      imgpath = os.path.join(tmppath, img)
      newpath = os.path.join(normal_path, img)
      shutil.move(imgpath, newpath)
    print("DONE")

###Split each disease folder into 70% training and 30% testing

In [16]:
for disease_folder in os.listdir(image_directory):
    disease_folder_path = os.path.join(image_directory, disease_folder)
    if os.path.isdir(disease_folder_path):
        # Create training and testing folders within the disease folder
        training_folder = os.path.join(disease_folder_path, 'training')
        os.makedirs(training_folder, exist_ok=True)
        testing_folder = os.path.join(disease_folder_path, 'testing')
        os.makedirs(testing_folder, exist_ok=True)

        # Get the list of files in the disease folder
        files = os.listdir(disease_folder_path)

        # Filter the files to include only the ones ending with ".jpg"
        images = [file for file in files if file.endswith(".jpg")]

        # Calculate the number of images for training and testing
        num_images = len(images)
        num_training_images = int(0.7 * num_images)
        num_testing_images = num_images - num_training_images

        # Randomly assign images to the training and testing folders
        random.shuffle(images)
        training_images = images[:num_training_images]
        testing_images = images[num_training_images:]

        # Move the training images to the training folder
        for image in training_images:
            image_path = os.path.join(disease_folder_path, image)
            new_image_path = os.path.join(training_folder, image)
            shutil.move(image_path, new_image_path)

        # Move the testing images to the testing folder
        for image in testing_images:
            image_path = os.path.join(disease_folder_path, image)
            new_image_path = os.path.join(testing_folder, image)
            shutil.move(image_path, new_image_path)

print("Images have been split into 70% training and 30% testing.")

Images have been split into 70% training and 30% testing.


In [17]:
classes = { 0: "Normal",
            1: "Diabetes",
            2: "Glaucoma",
            3: "Cataract",
            4: "Age related Macular Degeneration"
          }

###Move the data into general Training and Testing folders

In [18]:
# Create general training and testing folders
ODIR_path = '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset'
training_folder = os.path.join(ODIR_path, 'training')
os.makedirs(training_folder, exist_ok=True)

testing_folder = os.path.join(ODIR_path, 'testing')
os.makedirs(testing_folder, exist_ok=True)
print("The general training and testing folders have been created.")

The general training and testing folders have been created.


In [19]:
# Create sets that contain the paths of the training and testing folders inside all the diseases of interest
training_paths = []
training_paths = set(training_paths)
testing_paths = []
testing_paths = set(testing_paths)

for disease_folder in os.listdir(image_directory):
  disease_folder_path = os.path.join(image_directory, disease_folder)

  # Check if the disease is of our interest
  if disease_folder in list(classes.values()):
    # Move the training folders into the general training folder
    disease_training_path = os.path.join(disease_folder_path, 'training')
    new_disease_training_path = os.path.join(disease_folder_path, (disease_folder + '_training'))
    if os.path.isdir(disease_training_path):
      os.rename(disease_training_path, new_disease_training_path)
    if not os.path.isdir(new_disease_training_path):
      shutil.move(new_disease_training_path, training_folder)

    # Move the testing folders into the general testing folder
    disease_testing_path = os.path.join(disease_folder_path, 'testing')
    new_disease_testing_path = os.path.join(disease_folder_path, (disease_folder + '_testing'))
    if os.path.isdir(disease_testing_path):
      os.rename(disease_testing_path, new_disease_testing_path)
    if not os.path.isdir(new_disease_training_path):
      shutil.move(new_disease_testing_path, testing_folder)

print("The training and testing folders for each disease have been moved into the general training and testing folders.")

The training and testing folders for each disease have been moved into the general training and testing folders.


###Create a csv file that contains all the images of interest

In [20]:
# Create a list of all images file names

image_file_names = []
for folder in os.listdir(ODIR_path):
  if folder == 'training':
    training_folder = os.path.join(ODIR_path, 'training')
    for training_disease in os.listdir(training_folder):
      training_disease_folder = os.path.join(training_folder, training_disease)
      image_file_names.extend(os.listdir(training_disease_folder))

  elif folder == 'testing':
    testing_folder = os.path.join(ODIR_path, 'testing')
    for testing_disease in os.listdir(testing_folder):
      testing_disease_folder = os.path.join(testing_folder, testing_disease)
      image_file_names.extend(os.listdir(testing_disease_folder))

In [21]:
# Number of total images of interest
len(image_file_names)

5324

In [22]:
# Create a csv file for the images of interest
selected_rows = []
for index, row in processed_labels.iterrows():
    image_name = row['filename']

    # Check if the image file exists in the list of image file names
    if image_name in image_file_names:
        selected_rows.append(row)

# Create a new DataFrame with the selected rows
filtered_df = pd.DataFrame(selected_rows)

# Write the new DataFrame to a new CSV file
new_csv_file_path = os.path.join(ODIR_path, 'filtered.csv')
filtered_df.to_csv(new_csv_file_path, index=False)

print("The new csv file has been created successfully.")

The new csv file has been created successfully.


###Data Augmentation

In [51]:
# Check the balance of the classes
class_counts = filtered_df['class_name'].value_counts()
print(class_counts)

Normal                              2873
Diabetes                            1608
Cataract                             293
Glaucoma                             284
Age related Macular Degeneration     266
Name: class_name, dtype: int64


In [52]:
# Create folders for the augmented images in training and testing
augmented_training = os.path.join(ODIR_path, 'augmented_training')
augmented_testing = os.path.join(ODIR_path, 'augmented_testing')
os.makedirs(augmented_training, exist_ok=True)
os.makedirs(augmented_testing, exist_ok=True)

for cls in classes.values():
  disease_augmented_training = os.path.join(augmented_training, (cls + '_augmented_training'))
  disease_augmented_testing = os.path.join(augmented_testing, (cls + '_augmented_testing'))
  os.makedirs(disease_augmented_training, exist_ok=True)
  os.makedirs(disease_augmented_testing, exist_ok=True)

print("Folders for the augmented images in augmented training and augmented testing folders have been created.")


Folders for the augmented images in augmented training and augmented testing folders have been created.


In [53]:
# Find the class with maximum number of images
training_folder = '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/training'
testing_folder = '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/testing'
max_class = class_counts.idxmax()
max_class_training = os.path.join(training_folder, (max_class + '_training'))
max_class_testing = os.path.join(testing_folder, (max_class + '_testing'))

In [54]:
# Specify the parameters of data augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2)

In [55]:
# Get the size of the original images
sample_image_path = os.path.join(max_class_training, os.listdir(max_class_training)[0])
sample_image = cv2.imread(sample_image_path)
original_size = sample_image.shape[:2]  # Retrieve the height and width of the image

In [56]:
for cls in classes.values():
  # Augmentation in training
  disease_training = os.path.join(training_folder, (cls + '_training'))
  disease_augmented_training = os.path.join(augmented_training, (cls + '_augmented_training'))
  training_difference = len(os.listdir(max_class_training)) - len(os.listdir(disease_training))
  if training_difference > 0 and len(os.listdir(disease_augmented_training)) < training_difference:
    training_augmented_images = datagen.flow_from_directory(
            training_folder,
            target_size= original_size,
            batch_size = 1,
            classes = [(cls+'_training')],
            class_mode = None,
            save_to_dir = disease_augmented_training,
            save_prefix = ('augmented_' + cls + '_'),
            save_format = 'jpg',
            shuffle = False
            )
    for _ in range(training_difference):
      training_augmented_images.next()

print("Augmented images for training have been created.")

Augmented images for training have been created.


In [57]:
# Check the number of the augmented images for each class in training
for i in os.listdir(augmented_training):
  itrain = os.path.join(augmented_training, i)
  print(i + " = ", len(os.listdir(itrain)))

Normal_augmented_training =  0
Diabetes_augmented_training =  886
Glaucoma_augmented_training =  1813
Cataract_augmented_training =  1806
Age related Macular Degeneration_augmented_training =  1825


In [58]:
for cls in classes.values():
  #Augmentation in testing
  disease_testing = os.path.join(testing_folder, (cls + '_testing'))
  disease_augmented_testing = os.path.join(augmented_testing, (cls + '_augmented_testing'))
  testing_difference = len(os.listdir(max_class_testing)) - len(os.listdir(disease_testing))
  if testing_difference > 0 and len(os.listdir(disease_augmented_testing)) < testing_difference:
    testing_augmented_images = datagen.flow_from_directory(
            testing_folder,
            target_size = original_size,
            batch_size = 1,
            classes = [(cls+'_testing')],
            class_mode = None,
            save_to_dir = disease_augmented_testing,
            save_prefix = 'augmented_' + cls + '_',
            save_format ='jpg',
            shuffle = False
            )
    # Generate the specified number of augmented images
    for _ in range(testing_difference):
      testing_augmented_images.next()

print("Augmented images for testing have been created.")

Found 483 images belonging to 1 classes.
Augmented images for testing have been created.


In [59]:
# Check the number of the augmented images for each class in testing
for i in os.listdir(augmented_testing):
  itest = os.path.join(augmented_testing, i)
  print(i + " = ", len(os.listdir(itest)))

Normal_augmented_testing =  0
Glaucoma_augmented_testing =  776
Cataract_augmented_testing =  774
Age related Macular Degeneration_augmented_testing =  782
Diabetes_augmented_testing =  379


###Create a CSV file for the augmented images in training and testing

In [66]:
augmented_data = []

id_counter = 0
# Iterate over the folders in the main folder
for cls in classes.values():
  disease_augmented_training = os.path.join(augmented_training, (cls + '_augmented_training'))
  disease_augmented_testing = os.path.join(augmented_testing, (cls + '_augmented_testing'))

  # Add the augmented training images
  if os.path.isdir(disease_augmented_training):
    # Iterate over the images in each class folder
    for image_file in os.listdir(disease_augmented_training):
      # Create the target list based on the class_name
      target = [1 if i == cls else 0 for i in classes.values()]

      # Append the image data to the list
      augmented_data.append([id_counter, image_file, cls, target])

      # Increment the ID counter
      id_counter += 1

  # Add the augmented testing images
  if os.path.isdir(disease_augmented_testing):
    # Iterate over the images in each class folder
    for image_file in os.listdir(disease_augmented_testing):
      # Create the target list based on the class_name
      target = [1 if i == cls else 0 for i in classes.values()]

      # Append the image data to the list
      augmented_data.append([id_counter, image_file, cls, target])

      # Increment the ID counter
      id_counter += 1

# Creating a dataframe for the augmented images data
augmented_df = pd.DataFrame(augmented_data, columns=["ID", "filename", "class_name", "target"])

# Path to the CSV file to be created
augmented_csv_path = os.path.join(ODIR_path, 'augmented.csv')

augmented_df.to_csv(augmented_csv_path, index=False)

print("CSV file for the augmented images has been created.")

CSV file for the augmented images has been created.


In [67]:
augmented_df.head()

,ID,filename,class_name,target
0,0,augmented_Diabetes__0_4126838.jpg,Diabetes,"[0, 1, 0, 0, 0]"
1,1,augmented_Diabetes__1_5142113.jpg,Diabetes,"[0, 1, 0, 0, 0]"
2,2,augmented_Diabetes__2_5013863.jpg,Diabetes,"[0, 1, 0, 0, 0]"
3,3,augmented_Diabetes__3_2848479.jpg,Diabetes,"[0, 1, 0, 0, 0]"
4,4,augmented_Diabetes__4_1365883.jpg,Diabetes,"[0, 1, 0, 0, 0]"


In [68]:
augmented_df.shape

(9041, 4)

In [71]:
print("Total numbers in each class after augmentation: ")

for cls in classes.values():
  disease_training = os.path.join(training_folder, (cls + '_training'))
  disease_augmented_training = os.path.join(augmented_training, (cls + '_augmented_training'))
  total_training = len(os.listdir(disease_training)) + len(os.listdir(disease_augmented_training))
  print(cls + " training = ", total_training)

  disease_testing = os.path.join(testing_folder, (cls + '_testing'))
  disease_augmented_testing = os.path.join(augmented_testing, (cls + '_augmented_testing'))
  total_testing = len(os.listdir(disease_testing)) + len(os.listdir(disease_augmented_testing))
  print(cls + "testing = ", total_testing)


Total numbers in each class after augmentation: 
Normal training =  2011
Normaltesting =  862
Diabetes training =  2011
Diabetestesting =  862
Glaucoma training =  2011
Glaucomatesting =  862
Cataract training =  2011
Cataracttesting =  862
Age related Macular Degeneration training =  2011
Age related Macular Degenerationtesting =  862
